In [ ]:
## Test Team Formation framework with real datasets
## Balancing Task Coverage vs. Maximum Expert Load
## Karan Vombatkere, Spring 2022

#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json, time
import TeamFormationProblem as TFP

In [ ]:
bibs2020_covList = [90.72,163.62,217.13,250.47,273.55,290.12,304.03,317.9,328.97,339.57,349.45,357.32,365.48,373.27,380.27,386.68,392.17,397.12,402.12,
406.7,411.03,415.37,419.98,423.98,427.98,431.65,435.98,440.98,445.32,449.65,454.23,456.9,460.3,463.57,466.47,469.83,472.93,474.37,475.6,477.38,478.58,
479.42,480.42,481.67,482.0,482.92,483.75,484.83,486.0,486.67,487.33,488.0,489.0,489.67,490.17,490.83,491.5,492.17,492.83,493.17,493.5,493.83,494.17,
494.5,494.83,495.17,495.5,495.83,496.17,496.5,496.83,497.17,497.5,497.83,498.17,498.5,498.83,499.17,499.5,499.83,500.17,500.5,500.75,501.0,501.25,
501.45,501.65,501.85,502.05,502.05,502.05,502.05,502.05,502.05,502.05,502.05,502.05,502.05,502.05,502.05]

bibs2015_covList = [1636.06,2962.07,3976.15,4773.0,5345.78,5810.93,6201.01,6538.07,6816.47,7051.95,7253.95,7437.05,7577.0,7686.22,7783.92,7868.37,7937.9,
7996.32,8046.61,8094.57,8137.25,8175.47,8213.75,8251.69,8280.44,8305.07,8326.32,8345.87,8365.06,8382.97,8399.72,8414.84,8427.81,8437.85,8448.09,8458.94,
8468.02,8476.77,8486.41,8494.05,8500.45,8508.31,8516.33,8525.04,8530.95,8535.84,8540.65,8544.1,8548.18,8550.25]

bibs2010_covList = [16106.41,20314.17,20858.87,21085.92,21207.77,21265.34,21305.94,21330.59,21347.24,21358.95,21374.05]

imdb2020_covList = [2056.52,3838.52,5505.95,6638.51,7306.9,7640.89,7783.11,7791.44,7796.42,7800.77,7804.77,7808.77,7812.77,7816.77,7820.77,7824.77,7828.77,
7832.77,7836.77,7840.77,7844.77,7848.77,7852.77,7854.93,7856.27,7857.43,7858.0,7858.0,7858.0,7858.0,7858.0,7858.0,7858.0,7858.0,7858.0,7858.0,7858.0,7858.0,
7858.0,7858.0,7858.0,7858.0,7858.0,7858.0,7858.0,7858.0,7858.0,7858.0,7858.0,7858.0]

imdb2018_covList = [3661.5,6743.72,9526.9,11258.07,12233.38,12759.82,12986.41,13017.97,13022.57,13027.31,13030.31,13033.63,13036.63,13039.63,13042.63,13045.63,
13048.63,13051.63,13054.63,13057.63,13060.63,13063.63,13066.63,13069.63,13072.63,13075.63,13078.63,13081.63,13084.63,13087.63,13090.63,13093.63,13096.63,13099.63,
13102.63,13105.63,13108.63,13111.63,13114.63,13117.63,13120.63,13123.63,13126.63,13129.63,13132.63,13135.63,13138.63,13141.63,13144.63,13147.63]

imdb2015_covList = [5286.97,9765.69,13534.03,15750.51,16923.74,17604.37,17848.23,17866.78,17880.91,17894.89,17906.89,17918.72,17930.72,17943.72,17956.22,17968.72,
17981.22,17994.22,18006.72,18019.22,18031.72,18043.06,18055.56,18067.39,18078.89,18090.72,18098.72,18102.89,18106.25,18106.25,18106.25,18106.25,18106.25,18106.25,
18106.25,18106.25,18106.25,18106.25,18106.25,18106.25,18106.25,18106.25,18106.25,18106.25,18106.25,18106.25,18106.25,18106.25,18106.25,18106.25]

freelancer_covList = [592.78, 770.35, 853.98, 896.97, 927.43, 939.47, 944.88, 950.05, 954.1 , 958.07, 961.38, 965.37, 968.85, 972.1 , 974.1 , 976.4 ,
978.58, 980.87, 982.63, 984.12, 985.1 , 985.8 , 986.4 , 987.38, 988.32, 989.25, 990.1 , 990.95, 991.4 , 991.8 , 992.  , 992.  ,
992.  , 992.  , 992.  , 992.  , 992.  , 992.  , 992.  , 992.  , 992.  , 992.  , 992.  , 992.  , 992.  , 992.  , 992.  , 992.  , 992.  , 992.  ]

Guru_covList = [2759.97, 3068.83, 3133.5, 3152.19, 3161.93, 3168.25, 3172.03, 3174.32, 3177.28, 3179.17, 3181.28, 3182.78, 3184.14, 3185.58, 3186.79, 3187.90,
3188.81, 3189.60, 3190.26, 3190.85, 3191.40, 3191.87, 3192.20, 3192.54, 3192.85, 3193.08, 3193.27, 3193.27, 3193.27, 3193.27]


# lambdaList = [0.001,0.05, 0.1, 0.25, 0.5, 1, 2, 4]

In [ ]:
#Given list of coverages, and list of lambda values - find max for each lambda
def findBestLambda(coverageList, lambdaVals, plot_name, font_size):
    #Plot lambda curve of max coverages and max loads
    plt.figure(figsize=(8,6))

    lambdaMaxDict = {'lambdas': [], 'maxCoverages':[], 'workLoads': []}

    workload_vals = [10,30,50,70,90,110,130,150,170,190,210]

    for l_val in lambdaVals:
        F_prev = 0 #set F_prev to 0

        for i, c_i in enumerate(coverageList):
            thresh = i+1
            #thresh = workload_vals[i]
            F_val = (l_val * c_i) - thresh
            
            if F_val <= F_prev:
                lambdaMaxDict['lambdas'].append(l_val)
                lambdaMaxDict['maxCoverages'].append(coverageList[i-1])
                lambdaMaxDict['workLoads'].append(i)
                print("Max value found. Lambda={}, Max Coverage={:.2f}, Work Load={}".format(l_val, coverageList[i-1], i))
                break
            
            F_prev = F_val
        
        if F_val >= F_prev:
            # lambdaMaxDict['lambdas'].append(l_val)
            # lambdaMaxDict['maxCoverages'].append(coverageList[i])
            # lambdaMaxDict['workLoads'].append(thresh)
            print("Max value not found. Lambda={}, Max Coverage={:.2f}, Work Load={}".format(l_val, coverageList[i-1], i))
    
    # Plot the max values
    plt.plot(lambdaMaxDict['workLoads'], lambdaMaxDict['maxCoverages'], '--*', label=plot_name)
    plt.rc('xtick', labelsize=font_size)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=font_size)    # fontsize of the tick labels
    plt.rc('legend', fontsize=font_size)

    # title_text = 'Threshold Greedy varying Lambda (Guru)'
    # plt.title(title_text, fontsize=12)
    plt.xlabel('$L_{max}(A)$', fontsize=font_size)
    plt.ylabel('$C(A)$', fontsize=font_size)

    i=0
    # zip joins x and y coordinates in pairs
    for x,y in zip(lambdaMaxDict['workLoads'],lambdaMaxDict['maxCoverages']):
        label = "$\lambda={}$".format(lambdaVals[i])
        i += 1
        plt.annotate(label, # this is the text
                    (x,y), # these are the coordinates to position the label
                    textcoords="offset points", # how to position the text
                    xytext=(5,-12), # distance from text to points (x,y)
                    ha='center',
                    fontsize = font_size) # horizontal alignment can be left, right or center

    plt.legend(loc='lower right')
    plt.show()

In [ ]:
lambdaList_bibs2015 = [0.001, 0.005, 0.05, 0.1, 0.3]
lambdaList_bibs2020 = [0.05, 0.1, 0.2, 0.3, 1, 4]

lambdaList_freelancer = [0.005, 0.01, 0.1, 1, 4]
lambdaList_guru = [0.005, 0.05, 0.1, 0.5, 1, 4]

lambdaList_imdb2015 = [0.001, 0.05, 0.1, 2]
lambdaList_imdb2018 = [0.001, 0.005, 0.05, 0.3]
lambdaList_imdb2020 = [0.001, 0.1, 0.5, 2]

plottingList = [(bibs2015_covList, lambdaList_bibs2015, 'Bibsonomy-2015'), 
                (bibs2020_covList, lambdaList_bibs2020, 'Bibsonomy-2020'),
                (freelancer_covList, lambdaList_freelancer, 'Freelancer'),
                (Guru_covList, lambdaList_guru, 'Guru'),
                (imdb2015_covList, lambdaList_imdb2015, 'IMDB-2015'),
                (imdb2018_covList, lambdaList_imdb2018, 'IMDB-2018'),
                (imdb2020_covList, lambdaList_imdb2020, 'IMDB-2020')]

In [ ]:
for plotDetails in plottingList:
    findBestLambda(plotDetails[0], plotDetails[1], plot_name = plotDetails[2], font_size=14)

#### Freelancer Dataset

In [ ]:
#Import Freelancer data
#Freelancer from DropBox link: https://www.dropbox.com/sh/8zpsi1etvvvvj5k/AAD-J9ZQmSsbnSmEILBMD9uxa/datasets/real?dl=0&subfolder_nav_tracking=1
#freelance_experts.csv and freelance_projects.csv

def extract_skills(row):
    skills = []
    for i,val in enumerate(row):
        if val == 1:
            skills.append(str(i))
    return skills            

    
def importFreelancerData(experts_filename='datasets/freelancer/freelancer_experts.csv', tasks_filename='datasets/freelancer/freelancer_projects.csv'):
    #Extract tasks skills as list
    freelance_tasks_df = pd.read_csv(tasks_filename, header=None)
    print("Freelancer tasks df shape: ", freelance_tasks_df.shape)
    freelance_tasks_df['Task_Skills'] = freelance_tasks_df.apply(lambda row: extract_skills(row), axis=1)
    task_skills_list = freelance_tasks_df.Task_Skills.to_list()
    
    #Extract experts skills as list
    freelance_experts_df = pd.read_csv(experts_filename, header=None)
    print("Freelancer experts df shape: ", freelance_experts_df.shape)
    freelance_experts_df['Expert_Skills'] = freelance_experts_df.apply(lambda row: extract_skills(row), axis=1)
    expert_skills_list = freelance_experts_df.Expert_Skills.to_list()

    print("Imported Freelancer dataset. Num Experts={}, Num Tasks={}".format(len(expert_skills_list),len(task_skills_list)))

    return task_skills_list, expert_skills_list
    

In [ ]:
t,e = importFreelancerData()
FreelancerTest = TFP.TeamFormationProblem(t, e)

In [ ]:
runtimeDict, F_vals, workLoad_vals = FreelancerTest.computeTaskAssigment(algorithms=['random', 'no_update_greedy','task_greedy'], lambdaVal=0.1)

In [ ]:
# freelancerCovList = FreelancerTest.getCoverageValues()

#### Guru Dataset

In [ ]:
#Guru Dataset
def extract_skills_guru(row):
    skills = []
    for i,val in enumerate(row):
        if val == 1:
            skills.append(str(i))
    return skills 

def importGuruData(experts_filename='datasets/guru/guru_experts.csv', tasks_filename='datasets/guru/guru_tasks.csv'):
    #Extract tasks skills as list
    guru_tasks_df = pd.read_csv(tasks_filename, header=None)
    print("Guru tasks df shape: ", guru_tasks_df.shape)
    guru_tasks_df['Task_Skills'] = guru_tasks_df.apply(lambda row: extract_skills_guru(row), axis=1)
    task_skills_list = guru_tasks_df.Task_Skills.to_list()
    task_skills_list = task_skills_list[0:-1]
    
    #Extract experts skills as list
    guru_experts_df = pd.read_csv(experts_filename, header=None)
    print("Guru experts df shape: ", guru_experts_df.shape)
    guru_experts_df['Expert_Skills'] = guru_experts_df.apply(lambda row: extract_skills_guru(row), axis=1)
    expert_skills_list = guru_experts_df.Expert_Skills.to_list()
    expert_skills_list = expert_skills_list[0:-1]

    print("Imported Guru dataset. Num Experts={}, Num Tasks={}".format(len(expert_skills_list),len(task_skills_list)))

    return task_skills_list, expert_skills_list
    

In [ ]:
t,e = importGuruData()
GuruTest = TFP.TeamFormationProblem(t, e)

In [ ]:
# runtimeDict, F_vals, workLoad_vals = GuruTest.computeTaskAssigment(algorithms=['lazy_greedy', 'random', 'no_update_greedy','task_greedy'], lambdaVal=0.1)
runtimeDict, F_vals, workLoad_vals = GuruTest.computeTaskAssigment(algorithms=['lazy_greedy'], lambdaVal=0.1)

#### IMDB Datasets

In [ ]:
#Import IMDB Data
def importIMDBData(experts_filename, tasks_filename):
    with open(experts_filename, 'r') as f:
        expert_skills_list = json.loads(f.read())
    
    with open(tasks_filename, 'r') as f:
        task_skills_list = json.loads(f.read())

    print("Imported IMDB dataset. Num Experts={}, Num Tasks={}".format(len(expert_skills_list),len(task_skills_list)))

    return task_skills_list, expert_skills_list, 

#Run algorithm on IMDB datasets
def testIMDBDatasets(write_flag, algoList):
    imdb_data_path = 'datasets/imdb/'
    movieYears = [2015, 2018, 2020]

    if write_flag:
        runTimeStamp = str(time.strftime("%m-%d-%H:%M:%S", time.localtime(time.time())))
        imdb_outfilename = "experiments/imdb_" + runTimeStamp + ".txt"
        outfile_imdb = open(imdb_outfilename, "a")
        outfile_imdb.write("IMDB dataset Team-Formation Algorithms: {}\n".format(runTimeStamp))

    for y in movieYears:
        experts_file = imdb_data_path + 'imdb_experts_' + str(y) + '.txt'
        tasks_file = imdb_data_path + 'imdb_tasks_' + str(y) + '.txt'
        print("IMDB Dataset: {}, {}".format('imdb_experts_' + str(y), 'imdb_tasks_' + str(y)))

        imdb_tasks, imdb_experts = importIMDBData(experts_file, tasks_file)
        IMDBTest = TFP.TeamFormationProblem(imdb_tasks[0:600], imdb_experts[0:100])

        rt_dict, f_dict, workload_dict, coverageList = IMDBTest.computeTaskAssigment(algorithms=algoList, plot_flag=False)
        coverageListString = ""
        for c_i in coverageList:
            c_i_str = ", "+str(np.round(c_i, 2))
            coverageListString += c_i_str
        print(coverageListString)
        #Write output to file
        if write_flag:
            runInfo = "\nIMDB movieYear = {}, Experts = {}, Tasks = {}".format(str(y), str(IMDBTest.n), str(IMDBTest.m))
            outfile_imdb.write(runInfo)

            f_info = "\nAlgorithm Objectives (F_max): Lazy Greedy = {}; No-Update-Greedy = {}; Task Greedy = {}; Random = {};\
                ".format(f_dict['lazyGreedy'], f_dict['noUpdateGreedy'], f_dict['taskGreedy'], f_dict['random'])
            outfile_imdb.write(f_info)   

            wload_info = "\nAlgorithm optimal workloads: Lazy Greedy = {}; No-Update-Greedy = {}; Task Greedy = {}; Random = {};\
                ".format(workload_dict['lazyGreedy'], workload_dict['noUpdateGreedy'], workload_dict['taskGreedy'], workload_dict['random'])
            outfile_imdb.write(wload_info)   

            runtimeInfo = "\nAlgorithm Runtimes: Total = {:.3f}s; Lazy Greedy = {:.3f}s; No-Update-Greedy = {:.3f}s; Task Greedy = {:.3f}s; Random = {:.3f}s;\
                \n".format(rt_dict['total'], rt_dict['lazyGreedy'], rt_dict['noUpdateGreedy'], rt_dict['taskGreedy'], rt_dict['random'])
            outfile_imdb.write(runtimeInfo)

            outfile_imdb.write("\nCoverage List: {}".format(coverageListString))

    
    if write_flag:
        outfile_imdb.close()

    return None
    

In [ ]:
# testIMDBDatasets(write_flag=True, algoList=['lazy_greedy', 'random', 'no_update_greedy', 'task_greedy'])

In [ ]:
#Get coverage lists
imdb_data_path = 'datasets/imdb/'
y = 2018
experts_file = imdb_data_path + 'imdb_experts_' + str(y) + '.txt'
tasks_file = imdb_data_path + 'imdb_tasks_' + str(y) + '.txt'
print("IMDB Dataset: {}, {}".format('imdb_experts_' + str(y), 'imdb_tasks_' + str(y)))

imdb_tasks, imdb_experts = importIMDBData(experts_file, tasks_file)
IMDBTest = TFP.TeamFormationProblem(imdb_tasks, imdb_experts, max_workload_threshold=100)


In [ ]:
runtimeDict, F_vals, workLoad_vals = IMDBTest.computeTaskAssigment(algorithms=['random', 'no_update_greedy', 'task_greedy'], lambdaVal=0.05)
#runtimeDict, F_vals, workLoad_vals = IMDBTest.computeTaskAssigment(algorithms=['no_update_greedy'], lambdaVal=0.1)

In [ ]:
#covDict = IMDBCoverages.getStepCoverageValues()

In [ ]:
imdb_data_path = 'datasets/imdb/'
y = 2015
experts_file = imdb_data_path + 'imdb_experts_' + str(y) + '.txt'
tasks_file = imdb_data_path + 'imdb_tasks_' + str(y) + '.txt'
print("IMDB Dataset: {}, {}".format('imdb_experts_' + str(y), 'imdb_tasks_' + str(y)))

imdb_tasks, imdb_experts = importIMDBData(experts_file, tasks_file)
IMDBLambdaTest = TFP.TeamFormationProblem(imdb_tasks[:1000], imdb_experts[300:600], max_workload_threshold=100)

t_arr, f_dict, t_maxArr, f_maxArr = IMDBLambdaTest.testLambdaTaskAssignment(algorithms=['lazy_greedy'])


In [ ]:
#Plot F_i for different Lambda for Lazy Greedy
plt.figure(figsize=(9,6))
#for l_val in f_dict.keys():
    #plt.plot(t_arr, f_dict[l_val], label='Lambda={:.3f}'.format(l_val))

# Plot the max values
plt.plot(t_maxArr, f_maxArr, '--*', label='Max F_i')

# title_text = 'Lazy Greedy Performance by varying Lambda (IMDB_2015)'
# plt.title(title_text, fontsize=12)
plt.xlabel('Workload Threshold, T_i', fontsize=12)
plt.ylabel('Coverage, C(A)', fontsize=12)


lambda_arr = [0.3, 0.2, 0.5, 1, 2, 5, 10, 20, 50, 100]
i=0
# zip joins x and y coordinates in pairs
for x,y in zip(t_maxArr,f_maxArr):
    label = "{:.1f}".format(lambda_arr[i])
    i += 1
    plt.annotate(label, # this is the text
                 (x,y), # these are the coordinates to position the label
                 textcoords="offset points", # how to position the text
                 xytext=(0,10), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center

plt.legend(loc='lower right')
plt.show()

#### Bibsonomy Datasets

In [ ]:
#Import Bibsonomy datasets
def importBibsonomyData(experts_filename, tasks_filename):
    with open(experts_filename, 'r') as f:
        expert_skills_list = json.loads(f.read())
    
    with open(tasks_filename, 'r') as f:
        task_skills_list = json.loads(f.read())

    print("Imported Bibsonomy dataset. Num Experts={}, Num Tasks={}".format(len(expert_skills_list),len(task_skills_list)))

    return task_skills_list, expert_skills_list

#Run algorithm on Bibsonomy datasets
def testBibsonomyDatasets(write_flag, algoList):
    bibsonomy_data_path = 'datasets/bibsonomy/'
    movieYears = [2010, 2015, 2020]

    if write_flag:
        runTimeStamp = str(time.strftime("%m-%d-%H:%M:%S", time.localtime(time.time())))
        bibs_outfilename = "experiments/bibsonomy_" + runTimeStamp + ".txt"
        outfile_bibsonomy = open(bibs_outfilename, "a")
        outfile_bibsonomy.write("Bibsonomy dataset Team-Formation Algorithms: {}\n".format(runTimeStamp))

    for y in movieYears:
        experts_file = bibsonomy_data_path + 'bibsonomy_experts_' + str(y) + '.txt'
        tasks_file = bibsonomy_data_path + 'bibsonomy_tasks_' + str(y) + '.txt'
        print("\nBibsonomy Dataset: {}, {}".format('bibsonomy_experts_' + str(y), 'bibsonomy_tasks_' + str(y)))

        bib_tasks, bib_experts = importBibsonomyData(experts_file, tasks_file)
        BibsonomyTest = TFP.TeamFormationProblem(bib_tasks[0:500], bib_experts[0:200])

        rt_dict, f_dict, workload_dict = BibsonomyTest.computeTaskAssigment(algorithms=algoList, plot_flag=False)

        #Write output to file
        if write_flag:
            runInfo = "\nBibsonomy paperYear = {}, Experts = {}, Tasks = {}".format(str(y), str(BibsonomyTest.n), str(BibsonomyTest.m))
            outfile_bibsonomy.write(runInfo)

            f_info = "\nAlgorithm Objectives (F_max): Lazy Greedy = {}; No-Update-Greedy = {}; Task Greedy = {}; Random = {};\
                ".format(f_dict['lazyGreedy'], f_dict['noUpdateGreedy'], f_dict['taskGreedy'], f_dict['random'])
            outfile_bibsonomy.write(f_info)   

            wload_info = "\nAlgorithm optimal workloads: Lazy Greedy = {}; No-Update-Greedy = {}; Task Greedy = {}; Random = {};\
                ".format(workload_dict['lazyGreedy'], workload_dict['noUpdateGreedy'], workload_dict['taskGreedy'], workload_dict['random'])
            outfile_bibsonomy.write(wload_info)   

            runtimeInfo = "\nAlgorithm Runtimes: Total = {:.3f}s; Lazy Greedy = {:.3f}s; No-Update-Greedy = {:.3f}s; Task Greedy = {:.3f}s; Random = {:.3f}s;\
                \n".format(rt_dict['total'], rt_dict['lazyGreedy'], rt_dict['noUpdateGreedy'], rt_dict['taskGreedy'], rt_dict['random'])
            outfile_bibsonomy.write(runtimeInfo)
    
    if write_flag:
        outfile_bibsonomy.close()
    
    return None


In [ ]:
#testBibsonomyDatasets(write_flag=True, algoList=['lazy_greedy', 'random', 'no_update_greedy', 'task_greedy'])

In [ ]:
bibsonomy_data_path = 'datasets/bibsonomy/'
y=2015
experts_file = bibsonomy_data_path + 'bibsonomy_experts_' + str(y) + '.txt'
tasks_file = bibsonomy_data_path + 'bibsonomy_tasks_' + str(y) + '.txt'
print("\nBibsonomy Dataset: {}, {}".format('bibsonomy_experts_' + str(y), 'bibsonomy_tasks_' + str(y)))

bib_tasks, bib_experts = importBibsonomyData(experts_file, tasks_file)
BibsonomyTest = TFP.TeamFormationProblem(bib_tasks, bib_experts)

In [ ]:
runtimeDict, F_vals, workLoad_vals = BibsonomyTest.computeTaskAssigment(algorithms=['random', 'no_update_greedy', 'task_greedy'], lambdaVal=0.05)

In [ ]:
#Plot F_i for different Lambda for Lazy Greedy
# plt.figure(figsize=(9,6))
# for l_val in Fi_dict.keys():
#     plt.plot(T_arr, Fi_dict[l_val], label='Lambda={:.3f}'.format(l_val))

# # Plot the max values
# plt.plot(TMaxArr, FMaxArr, '--*', label='Max F_i')

# title_text = 'Lazy Greedy Performance by varying Lambda (Bibsonomy_2015)'
# plt.title(title_text, fontsize=12)
# plt.xlabel('Workload Threshold, T_i')
# plt.ylabel('F_i')
# plt.legend(loc='upper right')
# plt.show()

In [ ]:
# max_threshold_arr = [5,10,40,80,100,150,200]
# rev_rt_arr, reg_rt_arr = [],[]
# for thresh in max_threshold_arr:
#     FreelancerTest = TFP.TeamFormationProblem(t[0:200], e[0:200], max_workload_threshold=thresh)
#     rev_rt, reg_rt = FreelancerTest.compare_Methods()
#     rev_rt_arr.append(rev_rt)
#     reg_rt_arr.append(reg_rt)

#Plot Runtimes
# plt.figure(figsize=(9,6))
# plt.plot(max_threshold_arr, rev_rt_arr, label='Reverse Threshold Runtime')
# plt.plot(max_threshold_arr, reg_rt_arr, label='Regular Lazy Runtime')

# title_text = 'Reverse Threshold vs. Regular Lazy runtimes'
# plt.title(title_text, fontsize=11)
# plt.xlabel('Max Threshold, T_i')
# plt.ylabel('Runtime, s')
# plt.legend(loc='lower right')
# plt.show()

#FreelancerTest.compute_reverseThreshold()
#FreelancerTest.compareTest_Lazy_Stochastic_Assignments()